In [1]:
import logging
import os
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords
import numpy as np
english_stemmer = nltk.stem.SnowballStemmer('english')

In [2]:
import pandas as pd
jobs = pd.read_csv('../00-data/joblist_combined.csv')

In [3]:
jobs['description'].values

array(["Computer Vision Scientist/Engineer\r\r\n\r\r\nAt Expedition Technology, we push the boundaries of what is possible every day. Our engineers collaboratively solve the toughest problems in Computer Vision, Digital Signal Processing, Image Processing and more using Machine Learning/Deep Learning. We are a research-focused company whose mission it is to solve our customer's most pressing needs creatively and with cutting-edge algorithms. The solutions we develop enable autonomous operation, improve intelligence and simplify how things get done. We're doing this by building the most talented, multi-disciplinary research and engineering development team possible.\r\r\n\r\r\nAre you an experienced (mid to senior level) Computer Vision/Image Processing scientist or engineer? Do you enjoy working on hard problems in an environment that respects your independence and need for work-life balance? If so, Expedition might just be the right place for you.\r\r\n\r\r\nNote: US Citizenship is a 

In [4]:
stop_words=set(stopwords.words('english'))
def tags_from_text( reviews,
                    blacklist,
                    bigram_weight=2, 
                    trigram_weight=3,
                    ngram_range=[2,4],
                    stop_words=None):
    
    tv=CountVectorizer(ngram_range=ngram_range, stop_words=stop_words, encoding='utf8')
    tv.fit(reviews)
    
    review_vec = tv.transform(reviews)
    n_reviews = len(reviews)*1.0
    #print review_vec
    feats = tv.get_feature_names()
    #print feats
    counts = np.squeeze(np.asarray(review_vec.sum(axis=0)))
    feat_scores = {}
    dict = {'feature':[], 'score':[],'count':[]}
    #print counts
    for feat, count  in zip(feats, counts):
        ngram_score = 1
        if not(feat.endswith('skill') 
               or feat.endswith('skills')
               or feat.startswith('perform') 
               or feat.startswith('deliver')
               or feat.startswith('ability')
               or feat.startswith('avail')
               or feat.endswith('experience')
               or feat.startswith('demonstrate')
               or 'knowledge' in feat
               or 'licen' in feat 
               or 'educat' in feat
               or 'able' in feat
               or 'cert' in feat):
            continue
            
        # if len(np.intersect1d(['skill','skills'], feat.split()[-1])) == 0:
        #    continue
        if len(feat.split())>1:
            ngram_score = bigram_weight
        if len(feat.split())>2:
            ngram_score = trigram_weight
        if len(feat.split())>3:
            ngram_score = 27
            #print "***",  ngram_score, feat 
        score = count**ngram_score
    
        if len(np.intersect1d(blacklist, feat.split())) > 0:
            score = 0.
        if score> 0:
            feat_scores[feat] =  score
            dict['feature'] += [feat]
            dict['score'] += [score]
            dict['count'] += [count]
    return pd.DataFrame(dict)

In [5]:
blacklist = []
score_features = tags_from_text(jobs['description'].values.astype('U'), blacklist)

In [6]:
score_features.sort_values('score', ascending=False, inplace=True)
score_features['norm_score'] = score_features['score']/score_features.score.max()

In [7]:
score_features[score_features['count']>50].sort_values('count',ascending=False)

,feature,score,count,norm_score
69,ability to,260100,510,2.916101e-14
4936,knowledge of,75625,275,8.478667e-15
1955,communication skills,32761,181,3.672986e-15
5808,of experience,22801,151,2.556325e-15
335,able to,19044,138,2.135110e-15
9532,years of experience,2248091,131,2.520438e-13
5923,on experience,16384,128,1.836886e-15
4201,hands on experience,1771561,121,1.986179e-13
819,and experience,12544,112,1.406366e-15
9344,with experience,10201,101,1.143681e-15


In [8]:
score_features.sort_values('count',ascending=False)[['feature']].to_csv("./01-candidate_features.tsv", encoding='utf8', index=False, sep='\t')